### 코스피 수집 분석 연습
* http://www.paxnet.co.kr/stock/sise/KOSPI?wlog_rpax=KOSPI
* 팍스넷의 데이터 웹크롤링을 통한 코스피 일별 거래정보 획득

In [10]:
import sqlite3
import requests
from bs4 import BeautifulSoup

addr='http://www.paxnet.co.kr/stock/sise/siseJisuDataListAjax?searchType=kospi&currentPageNo=1&searchSort=&searchExchange=KSE&searchTerm=&searchUpDown='
result=requests.get(addr)
src=result.content
#print(result.content)

soup=BeautifulSoup(src,'html.parser')

In [11]:
dbname='paxstock.db'

In [12]:
### 초기 db 만들기
def maketable():
    with sqlite3.connect(dbname) as conn:
        cur = conn.cursor()
        sql = """
        create table kospi(IDX INTEGER primary key NOT NULL UNIQUE,
        EPRICE REAL, 
        SPRICE REAL, 
        HPRICE REAL, 
        LPRICE REAL, 
        UDOWN REAL, 
        UDRATE REAL, 
        QUANTITY INTEGER)
        """
        cur.execute(sql)
        conn.commit()
        cur.close()

In [13]:
#maketable()

In [14]:
def insertdb(data,cur):
    #with sqlite3.connect(dbname) as conn:
        #cur=conn.cursor()
    sql="""
    insert into kospi
    (IDX,EPRICE,SPRICE,HPRICE,LPRICE,UDOWN,UDRATE,QUANTITY) values(?,?,?,?,?,?,?,?)
    """
    
    sql_check = 'select * from kospi where IDX ="'+str(data[0])+'"'
    result_check = cur.execute(sql_check).fetchall()
    #print("result_check=",result_check)
    if result_check:
        print("error result_check=",result_check)
        #print("중복 에러"+str(tcode))
        pass
    else:
        cur.execute(sql,data)
    #conn.commit()
    #cur.close()
    

def check_title(t,cur):
    sql_title = 'select TIDX from titles where TNAME="'+str(t)+'"'
    result_id = cur.execute(sql_title).fetchall()
    if(result_id):
        print("해당 TITLE 있음")
        return result_id[0][0]
    else:
        print("해당 TITLE 없음")
        sql_title = 'insert into titles (TNAME) values("'+t+'")'
        cur.execute(sql_title)
        sql_title='select last_insert_rowid()'
        result_id = cur.execute(sql_title).fetchall()
        return result_id[0][0]
    

In [15]:
def Mycrawl(soup):
    with sqlite3.connect(dbname) as conn:
        cur = conn.cursor()
        #sql = 'insert into movie (IDX, TITLE, POINT, CONTENT, TCODE) values(?,?,?,?,?)'
        res = soup.select('.table-scroll .table-data tbody > tr')
        
        for r in res:
            date_res = int(r.select(' td > strong')[0].get_text().replace('.',''))
            #print(date_res)
            idx_res = r.select('td')[0].get_text()
            jong_res = float(r.select('td.a-right')[0].get_text().replace(',',''))
            si_res = float(r.select('td.a-right')[1].get_text().replace(',',''))
            high_res = float(r.select('td.a-right')[2].get_text().replace(',',''))
            low_res = float(r.select('td.a-right')[3].get_text().replace(',',''))

            updown = r.select('td.a-right > span.rise')
            if(updown):
                #print("high updown=",updown)
                #print("high updown[0]=",updown[0])
                #print("high updown[0].get_text()=",updown[0].get_text())
                udcode = 1
                updown = float(updown[0].get_text().replace('상향','').strip())
                #print("updown=",updown)
            else:
                updown = r.select('td.a-right > span.fall')
                if(updown):
                    udcode = -1
                    updown = r.select('td.a-right > span.fall')
                    #print("low updown=",updown)
                    #print("lodw updown[0]=",updown[0].get_text())
                    updown =  float(updown[0].get_text().replace('하향','').strip())
                    #print("updown=",updown)
                else:
                    udcode = 0
                    #updown = r.select('td.a-right > span.fall')
                    #print("low updown=",updown)
                    #print("lodw updown[0]=",updown[0].get_text())
                    updown =  0.0
                    #print("updown=",updown)
                
            udrate=float(r.select("td.a-right")[5].get_text().replace('%','').strip())
            quantity=int(r.select("td.a-right")[6].get_text().replace(',','').strip())
            data=[date_res,jong_res,si_res,high_res,low_res,udcode*updown,udrate,quantity]

        #print("data=",data)
        #cur.execute(sql, data)
            insertdb(data, cur)
            
        conn.commit()
        cur.close()


In [16]:
def MyPrs(fr=0,to=50):
    for i in range(fr,to):
        url='http://www.paxnet.co.kr/stock/sise/siseJisuDataListAjax?searchType=kospi&currentPageNo='
        url+=str(i)+'&searchSort=&searchExchange=KSE&searchTerm=&searchUpDown='
        result=requests.get(url)
        src = result.content
        soup = BeautifulSoup(src,'html.parser')
        print("i = ",i)
        print("*"*50)
        Mycrawl(soup)
        print("*"*50)
        #print(url,end="\n")

In [17]:
MyPrs()

i =  0
**************************************************
error result_check= [(20190710, 2058.78, 2061.55, 2068.2, 2056.48, 6.75, 0.33, 355677)]
error result_check= [(20190709, 2052.03, 2070.35, 2075.3, 2052.03, -12.14, -0.59, 440122)]
error result_check= [(20190708, 2064.17, 2091.91, 2092.54, 2063.78, -46.42, -2.2, 515950)]
error result_check= [(20190705, 2110.59, 2111.0, 2115.43, 2096.23, 1.86, 0.09, 459391)]
error result_check= [(20190704, 2108.73, 2102.36, 2108.8, 2088.97, 12.71, 0.61, 446932)]
error result_check= [(20190703, 2096.02, 2120.75, 2127.71, 2093.32, -26.0, -1.23, 522709)]
error result_check= [(20190702, 2122.02, 2122.07, 2130.86, 2112.52, -7.72, -0.36, 460830)]
error result_check= [(20190701, 2129.74, 2147.24, 2147.24, 2126.87, -0.88, -0.04, 412291)]
error result_check= [(20190628, 2130.62, 2135.06, 2139.24, 2123.97, -3.7, -0.17, 537274)]
error result_check= [(20190627, 2134.32, 2120.73, 2141.27, 2118.22, 12.47, 0.59, 644900)]
error result_check= [(20190626, 2121.85, 2